# Aposteriori Unimodality applications

## Real life data

### Preprocessing

In [1]:
%load_ext autoreload 
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path

import aposteriori

import re
import hashlib

In [2]:
df = pd.read_pickle("data/attitudes_embedded.csv")
df

,tweet,toyou,toany,annotatorMinority,bert,annotatorPolitics,traditionalism,annotatorRace,annotatorAge,annotatorGender,freeSpeech,harmHateSpeech,intent,lingPurism,racism,racist
0,""" you ugly bitch!!!""- that seems to be my real...","[2, 1, 1, 2]","[3, 3, 1, 2]","[{}, pansexual, disabled, trans, {}, {}]","[0.0047034588642418385, 0.11112377792596817, 0...","[-1.0, -1.0, 1.0, 0.5]","[3, 1, 5, 4]","[black, black, white, white]","[25.0, 35.0, 50.0, 25.0]","[man, man, woman, woman]","[1, 1, 1, 2]","[4, 5, 5, 2]","[1, 1, 1, 2]","[1, 1, 5, 5]","[1, 1, 1, 4]","[1, 1, 1, 1]"
1,"""""""I am happy to report that drowning does kil...","[4, 2, 1, 2, 1, 1]","[4, 2, 1, 2, 2, 1]","[{}, {}, {}, {}, {}, {}]","[0.35706740617752075, 0.032704465091228485, 0....","[-0.5, -0.5, 0.5, 0.5, -1.0, -1.0]","[4, 3, 4, 3, 1, 4]","[black, black, white, white, white, white]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[woman, woman, woman, man, man, man]","[1, 1, 3, 4, 1, 1]","[4, 4, 3, 1, 5, 4]","[4, 2, 1, 3, 1, 1]","[2, 5, 5, 5, 5, 5]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 1, 1, 1, 2]","[2, 1, 1, 1, 1, 3]","[{}, black; female, nan, {}, {}, {}]","[-0.022276882082223892, 0.17076537013053894, 0...","[-0.5, -1.0, 1.0, 0.0, -1.0, -0.5]","[3, 4, 3, 5, 1, 3]","[black, black, white, white, white, white]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[woman, woman, man, woman, man, woman]","[1, 1, 5, 1, 1, 1]","[4, 5, 1, 5, 5, 4]","[1, 1, 1, 1, 1, 1]","[5, 4, 5, 5, 5, 4]","[1, 1, 5, 1, 1, 1]","[1, 1, 1, 1, 1, 1]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 2, 1, 1, 1, 1]","[1, 4, 1, 1, 1, 1]","[{}, black; female, nan, {}, {}, nan]","[0.07474598288536072, 0.24487990140914917, 0.2...","[0.0, -1.0, 1.0, 0.5, -0.5, -1.0]","[4, 4, 3, 4, 5, 5]","[black, black, white, white, white, white]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[man, woman, man, woman, woman, woman]","[1, 1, 5, 3, 1, 1]","[4, 5, 1, 3, 5, 5]","[1, 1, 1, 1, 1, 1]","[4, 4, 5, 5, 4, 3]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 3, 3, 4]","[1, 2, 4, 4, 4]","[{}, NONE, No i do not, none, {}]","[0.050774428993463516, 0.08187347650527954, 0....","[-1.0, 1.0, 0.5, 0.5, -1.0]","[5, 3, 5, 5, 5]","[black, black, white, white, white]","[45.0, 45.0, 45.0, 60.0, 40.0]","[man, man, woman, woman, woman]","[1, 5, 1, 1, 1]","[5, 4, 3, 5, 2]","[1, 3, 1, 3, 5]","[5, 3, 4, 5, 5]","[1, 3, 5, 1, 1]","[1, 4, 2, 2, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 2, 3]","[4, 5, 2, 3]","[pansexual, disabled, trans, {}, none, {}]","[0.22457970678806305, 0.3840716481208801, -0.4...","[-1.0, 1.0, 0.5, 0.5]","[1, 5, 5, 5]","[black, black, white, white]","[35.0, 30.0, 45.0, 65.0]","[man, woman, man, woman]","[1, 5, 1, 1]","[5, 5, 5, 5]","[1, 5, 1, 3]","[1, 5, 5, 5]","[1, 5, 1, 1]","[1, 1, 1, 1]"
622,worst part about getting highlights is sitting...,"[1, 3, 1, 1, 1, 1]","[1, 5, 1, 1, 1, 1]","[{}, NONE, {}, {}, {}, none]","[0.1781517118215561, 0.02407737821340561, 0.13...","[-1.0, 1.0, 0.5, 0.5, -0.5, -0.5]","[5, 3, 5, 2, 4, 3]","[black, black, white, white, white, white]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[man, man, woman, man, man, man]","[1, 5, 1, 2, 1, 1]","[5, 4, 4, 2, 3, 5]","[1, 3, 1, 1, 1, 1]","[5, 3, 5, 2, 5, 4]","[1, 5, 2, 3, 2, 1]","[1, 4, 1, 1, 1, 1]"
623,yids are not as honest as the rest of us,"[5, 5, 4, 5, 4, 1]","[5, 5, 4, 5, 4, 1]","[nan, {}, {}, none, {}, bisexual]","[0.2983386516571045, 0.31099164485931396, -0.4...","[-0.5, -0.5, 0.5, 0.5, -0.5, -0.5]","[3, 5, 2, 5, 4, 3]","[black, black, white, white, white, white]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[man, man, man, woman, woman, woman]","[1, 1, 5, 1, 2, 1]","[3, 3, 2, 5, 3, 5]","[5, 5, 4, 5, 5, 1]","[5, 3, 5, 5, 4, 5]","[1, 1, 4, 1, 2, 1]","[5, 3, 3, 5, 5, 1]"
624,"you've got to be fucking joking, my room has l...","[3, 3, 1, 1, 2, 1]","[2, 4, 1, 2, 3, 2]","[

In [3]:
df = df.loc[
    :,
    [
        "tweet",
        "racism",
        "annotatorAge",
        "annotatorRace",
        "annotatorGender",
    ],
]
df

,tweet,racism,annotatorAge,annotatorRace,annotatorGender
0,""" you ugly bitch!!!""- that seems to be my real...","[1, 1, 1, 4]","[25.0, 35.0, 50.0, 25.0]","[black, black, white, white]","[man, man, woman, woman]"
1,"""""""I am happy to report that drowning does kil...","[1, 1, 5, 5, 1, 1]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[black, black, white, white, white, white]","[woman, woman, woman, man, man, man]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 5, 1, 1, 1]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[black, black, white, white, white, white]","[woman, woman, man, woman, man, woman]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 1, 5, 5, 1, 1]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[black, black, white, white, white, white]","[man, woman, man, woman, woman, woman]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 5, 1, 1]","[45.0, 45.0, 45.0, 60.0, 40.0]","[black, black, white, white, white]","[man, man, woman, woman, woman]"
...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 1, 1]","[35.0, 30.0, 45.0, 65.0]","[black, black, white, white]","[man, woman, man, woman]"
622,worst part about getting highlights is sitting...,"[1, 5, 2, 3, 2, 1]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[black, black, white, white, white, white]","[man, man, woman, man, man, man]"
623,yids are not as honest as the rest of us,"[1, 1, 4, 1, 2, 1]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[black, black, white, white, white, white]","[man, man, man, woman, woman, woman]"
624,"you've got to be fucking joking, my room has l...","[1, 4, 1, 3, 3, 1]","[40.0, 25.0, 40.0, 35.0, 45.0, 35.0]","[black, black, white, white, white, white]","[woman, woman, woman, woman, woman, woman]"


Clear out NaNs from dataset

In [4]:
nan_age = df.annotatorAge.apply(lambda x: np.any(np.isnan(x)))
nan_race = df.annotatorRace.apply(lambda x: "na" in x)
nan_gender = df.annotatorGender.apply(lambda x: "na" in x)

nan_age.sum(), nan_race.sum(), nan_gender.sum()

(41, 41, 39)

In [5]:
def find_inconsistent_rows(df):
    inconsistent_rows = []

    for index, row in df.iterrows():
        list_lengths = [
            len(row[col]) for col in df.columns if isinstance(row[col], list)
        ]

        if len(set(list_lengths)) > 1:  # Check if lengths are inconsistent
            inconsistent_rows.append(index)

    return df.loc[inconsistent_rows]


find_inconsistent_rows(df)

,tweet,racism,annotatorAge,annotatorRace,annotatorGender


In [6]:
df.annotatorRace = df.annotatorRace.apply(lambda x: None if ("na" in x) else x)
df.annotatorGender = df.annotatorGender.apply(
    lambda x: None if ("na" in x) else x
)
df.annotatorAge = df.annotatorAge.apply(
    lambda x: None if np.any([np.isnan(age) for age in x]) else x
)
df.annotatorAge = df.annotatorAge.apply(
    lambda x: [int(age) for age in x] if x is not None else None
)
df["random"] = df.racism.apply(
    lambda x: [np.random.randint(1, 3) for _ in range(len(x))]
)
df = df.dropna()
df

,tweet,racism,annotatorAge,annotatorRace,annotatorGender,random
0,""" you ugly bitch!!!""- that seems to be my real...","[1, 1, 1, 4]","[25, 35, 50, 25]","[black, black, white, white]","[man, man, woman, woman]","[2, 1, 2, 2]"
1,"""""""I am happy to report that drowning does kil...","[1, 1, 5, 5, 1, 1]","[40, 30, 40, 30, 25, 25]","[black, black, white, white, white, white]","[woman, woman, woman, man, man, man]","[2, 1, 2, 1, 1, 1]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 5, 1, 1, 1]","[30, 40, 35, 40, 25, 45]","[black, black, white, white, white, white]","[woman, woman, man, woman, man, woman]","[2, 2, 2, 2, 2, 1]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 1, 5, 5, 1, 1]","[35, 40, 35, 40, 35, 45]","[black, black, white, white, white, white]","[man, woman, man, woman, woman, woman]","[2, 1, 2, 1, 1, 1]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 5, 1, 1]","[45, 45, 45, 60, 40]","[black, black, white, white, white]","[man, man, woman, woman, woman]","[1, 2, 2, 2, 2]"
...,...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 1, 1]","[35, 30, 45, 65]","[black, black, white, white]","[man, woman, man, woman]","[1, 1, 1, 1]"
622,worst part about getting highlights is sitting...,"[1, 5, 2, 3, 2, 1]","[45, 45, 35, 35, 25, 50]","[black, black, white, white, white, white]","[man, man, woman, man, man, man]","[2, 1, 2, 2, 2, 1]"
623,yids are not as honest as the rest of us,"[1, 1, 4, 1, 2, 1]","[30, 30, 40, 60, 35, 40]","[black, black, white, white, white, white]","[man, man, man, woman, woman, woman]","[2, 1, 2, 2, 1, 1]"
624,"you've got to be fucking joking, my room has l...","[1, 4, 1, 3, 3, 1]","[40, 25, 40, 35, 45, 35]","[black, black, white, white, white, white]","[woman, woman, woman, woman, woman, woman]","[2, 2, 1, 1, 1, 2]"


### Results

In [7]:
def extract_annotations_and_attributes(
    df: pd.DataFrame, value_col: str, feature_col: str, comment_key_col: str
) -> tuple[list, list]:
    all_annotations = []
    all_attributes = []
    all_keys = []

    for _, row in df.iterrows():
        values = row[value_col]
        factors = row[feature_col]
        key = row[comment_key_col]
        assert len(values) == len(
            factors
        ), f"values {values} (length {len(values)}) \n"
        f"have different length than factors{factors} (length {len(factors)})"
        all_annotations.extend(values)
        all_attributes.extend(factors)
        # extend the key for each value in the above extracted list
        all_keys.extend([row[comment_key_col]] * len(factors))

    return all_annotations, all_attributes, all_keys


def run_aposteriori(
    df: pd.DataFrame,
    value_col: str,
    feature_col: str,
    comment_key_col: str,
    bins: int = -1
) -> pd.Series:
    if bins == -1:
        bins = len(np.unique(df[value_col]))
    annotations, attributes, keys = extract_annotations_and_attributes(
        df=df,
        value_col=value_col,
        feature_col=feature_col,
        comment_key_col=comment_key_col,
    )

    stat = aposteriori.aposteriori_unimodality(
        annotations=annotations,
        factor_group=attributes,
        comment_group=keys,
        bins=bins,
    )
    return pd.Series(stat)

In [8]:
print(
    run_aposteriori(
        df,
        feature_col="annotatorAge",
        value_col="racism",
        comment_key_col="tweet",
    )
)
print(
    run_aposteriori(
        df,
        feature_col="annotatorGender",
        value_col="racism",
        comment_key_col="tweet",
    )
)
print(
    run_aposteriori(
        df,
        feature_col="annotatorRace",
        value_col="racism",
        comment_key_col="tweet",
    )
)
print(
    run_aposteriori(
        df,
        feature_col="random",
        value_col="racism",
        comment_key_col="tweet",
    )
)

25    1.707755e-33
35    3.708877e-05
50    1.302804e-29
18    3.964538e-01
30    3.309262e-08
40    1.249250e-25
45    9.280764e-20
55    2.581551e-01
60    1.770396e-07
65    1.172850e-11
dtype: float64
man          1.000000e+00
woman        2.115972e-08
nonBinary    2.591290e-07
dtype: float64
black            1.000000e+00
white            1.000000e+00
hisp             2.593647e-08
middleEastern             NaN
other            6.227618e-03
dtype: float64
1    1.0
2    1.0
dtype: float64


/home/dimits/anaconda3/envs/std/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1081: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/dimits/anaconda3/envs/std/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1081: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


## Synthetic data

In [9]:
def str_list_to_list(str_list: str) -> list[str]:
    ls = str_list[1:-1].split(",")
    ls = [x.strip() if x is not None else x for x in ls]
    return ls


syn_df = pd.read_csv("data/synthetic.csv")
syn_df["comment_key"] = syn_df.message + syn_df.conv_id

syn_df.Toxicity = syn_df.Toxicity.apply(str_list_to_list).apply(
    lambda x: [int(tox) for tox in x]
)
syn_df.age_annot = syn_df.age_annot.apply(str_list_to_list)
syn_df.sex_annot = syn_df.sex_annot.apply(str_list_to_list)
syn_df.demographic_group_annot = syn_df.demographic_group_annot.apply(
    str_list_to_list
)
syn_df

,conv_id,message,intent,Toxicity,Argument Quality,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot,comment_key
0,028f869d-cf12-444a-aefc-62f8b6294acd,@FitnessJunkie07 Respect and honesty are unive...,"['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1]","['25', '25', '25', '25', '25', '25', '25', '25...","['male', 'male', 'male', 'male', 'male', 'male...","['Bisexual', 'Bisexual', 'Bisexual', 'Bisexual...","['Asian', 'Asian', 'Asian', 'Asian', 'Asian', ...","['Software Developer', 'Software Developer', '...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","['27', '21', '28', '65', '55', '60', '38', '40...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...",@FitnessJunkie07 Respect and honesty are unive...
1,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I agree, but sometimes chang...","['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","['27', '27', '27', '27', '27', '27', '27', '27...","['female', 'female', 'female', 'female', 'fema...","['Lesbian', 'Lesbian', 'Lesbian', 'Lesbian', '...","['Arab', 'Arab', 'Arab', 'Arab', 'Arab', 'Arab...","['Poet', 'Poet', 'Poet', 'Poet', 'Poet', 'Poet...","[""Master's"", ""Master's"", ""Master's"", ""Master's...","['27', '21', '28', '65', '55', '60', '38', '40...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I agree, but sometimes chang..."
2,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I completely agree that resp...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","['27', '21', '28', '65', '55', '60', '38', '40...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I completely agree that resp..."
3,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I understand where you're co...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2]","[2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","['27', '21', '28', '65', '55', '60', '38', '40...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I understand where you're co..."
4,028f869d-cf12-444a-aefc-62f8b6294acd,@LoyalVolunteer54 @FitnessJunkie07 It's import...,"['Moderator', 'Mod

In [10]:
find_inconsistent_rows(syn_df)

,conv_id,message,intent,Toxicity,Argument Quality,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot,comment_key


In [11]:
print(
    run_aposteriori(
        syn_df,
        feature_col="age_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)
print(
    run_aposteriori(
        syn_df,
        feature_col="sex_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)
print(
    run_aposteriori(
        syn_df,
        feature_col="demographic_group_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)

'21'    2.366598e-234
'27'    2.366598e-234
'28'    2.366598e-234
'32'    2.366598e-234
'38'    2.366598e-234
'40'    2.366598e-234
'45'    2.366598e-234
'50'    2.366598e-234
'55'    2.366598e-234
'60'    2.366598e-234
'65'    2.366598e-234
dtype: float64
'female'         1.746674e-03
'male'           7.134139e-01
'non-binary'    6.454357e-235
dtype: float64
'Arab'                1.936307e-234
'Black'               1.936307e-234
'Caucasian'            1.000000e+00
'Indigenous'          1.936307e-234
'Latino'              1.936307e-234
'Middle Eastern'      1.936307e-234
'Pacific Islander'    1.936307e-234
'South Asian'         1.936307e-234
'White'                1.000000e+00
dtype: float64


## Effect of bin size

In [12]:
for bins in range(1, 20):
    print(
        bins,
        run_aposteriori(
            df,
            feature_col="annotatorAge",
            value_col="racism",
            comment_key_col="tweet",
            bins=bins
        ),
    )

1 25   NaN
35   NaN
50   NaN
18   NaN
30   NaN
40   NaN
45   NaN
55   NaN
60   NaN
65   NaN
dtype: float64
2 25   NaN
35   NaN
50   NaN
18   NaN
30   NaN
40   NaN
45   NaN
55   NaN
60   NaN
65   NaN
dtype: float64
3 25    5.279175e-17
35    1.000000e+00
50    9.064921e-11
18    1.000000e+00
30    1.000000e+00
40    5.059598e-02
45    9.096963e-01
55    9.453361e-01
60    1.542267e-02
65    1.024865e-06
dtype: float64
4 25    5.357021e-28
35    1.000000e+00
50    1.766651e-24
18    3.964538e-01
30    5.291039e-02
40    8.048220e-11
45    1.058793e-04
55    3.401083e-01
60    6.169686e-07
65    3.279910e-07
dtype: float64
5 25    2.227540e-32
35    8.056433e-02
50    1.517690e-30
18    1.000000e+00
30    2.653637e-05
40    1.027556e-19
45    1.280582e-10
55    3.393591e-01
60    2.622263e-07
65    1.172850e-11
dtype: float64
6 25    9.109696e-34
35    7.228929e-03
50    2.088855e-30
18    3.964538e-01
30    7.241049e-06
40    7.548231e-22
45    8.717574e-15
55    3.393591e-01
60    1.115